In [1]:
from pulp import *
import max_weight_cap
import labTA
from schedule import Schedule
import stats
import swap
import past_exp_reader
import gspread
import copy
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

HOURS_LIMIT = 4 #limit of hours a TA can work
STUD_SLOTS_WORKED_CAP = 2
MAX_VALUE = 10
MIN_VALUE = 0
OVERLAPS = {'Sa_4': 'Sa_3', 'Sa_5':'Sa_4', 'Su_6':'Su_5', 'Su_7':'Su_6', 'Su_8':'Su_7', 'Su_9':'Su_8'} #dict of slots to check as keys, and overlapping slots as values
# slots and num of TA's desired
slotdict = {"M_7" : 8, "M_9" : 6,"Tu_7" : 5, "Tu_9" : 4,"W_7" : 4, "W_9" : 4,"Th_7" : 4, "Th_9" : 4,"F_7" : 4, "F_9" : 4,"Sa_3" : 5, "Sa_4" : 6,"Sa_5" : 5,"Su_5" : 4,"Su_6" : 3,"Su_7" : 6,"Su_8" : 4, "Su_9" : 6}
NUM_SLOTS = 16.0 #number of slots
NUM_STUDENTS = 45
MAX_HAP = 258.0
score = 3

In [2]:
import pandas as pd

In [3]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('labTA-match-secret.json', scope)
client = gspread.authorize(creds)

# Find workbook and open the first sheet
sheet = client.open('LabTA_test2').sheet1

# make dataframe
df_shir = pd.DataFrame(sheet.get_all_records())

#get experience dict
students = []
for stud_num in range(NUM_STUDENTS):
    name = df_shir.at[int(stud_num), "name"]
    students.append(name)
exp_dict = past_exp_reader.get_exp('historical_data.csv', students)

In [4]:
#shirley's schedule
shirley_data = {"M_7" : ['Tajreen Ahmed', 'Urvashi Uberoy', 'Ze-Xin Koh', 'Kyle Johnson', 'Ariel Rakovitsky', 'Caroline di Vittorio', 'Khyati Agrawal', 'Annie Zhou'], "M_9" : ['Cathleen Kong', 'HJ Suh', 'Ze-Xin Koh', 'Akash Pattnaik', 'Ariel Rakovitsky', 'Caroline di Vittorio'],"Tu_7" : ['Uri Schwartz','Alan Ding','Urvashi Uberoy','Akash Pattnaik','Bobby Morck'], "Tu_9" : ['Justin Chang','Alan Ding','Caio Costa','Bobby Morck'],"W_7" : ['Michelle Woo','Avi Bendory','Kawin Tiyawattanaroj','Tajreen Ahmed'], "W_9" : ['Michelle Woo','Avi Bendory','Kawin Tiyawattanaroj','Khyati Agrawal'],"Th_7" : ['Charlie Smith','Niranjan Shankar','Caio Costa','Ryan Golant'], "Th_9" : ['Charlie Smith','Arjun Devraj','Somya Arora','Jason Xu'],"F_7" : ['Annie Zhou','Nathan Alam','Sahan Paliskara','Connie Miao'], "F_9" : ['Somya Arora','Nathan Alam','Sahan Paliskara','Ryan Golant'],"Sa_3" : ['Anu Vellore','Ibrahim Ali Hashmi','Aditya Kohli','Lily Zhang','Ezra Zinberg'], "Sa_4" : ['Jackson Deitelzweig','Donovan Coronado','Jason Xu','Uri Schwartz','Ally Dalman','Catherine Yu'],"Sa_5" : ['Anu Vellore','Ibrahim Ali Hashmi','Connie Miao','Lily Zhang','Ezra Zinberg'],"Su_5" : ['Nala Sharadjaya','Arjun Devraj','Donovan Coronado','Niranjan Shankar'],"Su_6" : ['Kyle Johnson','Sandun Bambarandage','Jackson Deitelzweig'],"Su_7" : ['Yashodhar Govil','Shirley Z.','Aniela Macek','Chuk Uzoegwu','Nala Sharadjaya','Aditya Kohli'],"Su_8" : ['Cathleen Kong','Sandun Bambarandage','HJ Suh','Ally Dalman'], "Su_9" : ['Yashodhar Govil','Shirley Z.','Aniela Macek','Chuk Uzoegwu','Justin Chang','Catherine Yu']}
shirley_sched = Schedule(shirley_data)
print("Shirley schedule stats:")
stats.exp_stats(exp_dict, shirley_sched)
shir_hap = stats.sched_happiness(df_shir, shirley_sched)
print('Total Happiness: ', shir_hap[0])
print()
print('Availability to happiness correlation: ', shir_hap[1])
print()
print('Variance of happiness: ', shir_hap[2])
print()
print('Envy stats: ', shir_hap[3])
print()
print('Incorrect stats: ', shir_hap[4])
print()

Shirley schedule stats:
M_7  has average experience:  2.5
M_9  has average experience:  2.0
Tu_7  has average experience:  3.0
Tu_9  has average experience:  2.0
W_7  has average experience:  2.25
W_9  has average experience:  2.25
Th_7  has average experience:  2.0
Th_9  has average experience:  1.5
F_7  has average experience:  2.0
F_9  has average experience:  2.0
Sa_3  has average experience:  1.8
Sa_4  has average experience:  3.0
Sa_5  has average experience:  1.6
Su_5  has average experience:  2.5
Su_6  has average experience:  2.6666666666666665
Su_7  has average experience:  2.3333333333333335
Su_8  has average experience:  3.0
Su_9  has average experience:  2.3333333333333335
Total Happiness:  0.9031007751937985

Availability to happiness correlation:  -0.0173767262022999

Variance of happiness:  1.9222222222222216

Envy stats:  148

Incorrect stats:  63



In [5]:
df_aaron = pd.DataFrame(sheet.get_all_records())

ordered_slotdict = labTA.get_order(df_aaron)
blank_sched = Schedule()
aaron_sched = labTA.scheduler(df_aaron, score, ordered_slotdict, blank_sched)

unhap_studs = swap.get_unhappy(df_aaron)
swap_dict = swap.check_swap(df_aaron, aaron_sched, unhap_studs)
swap.correct_swap(df_aaron, aaron_sched, unhap_studs, swap_dict)
# print(df_aaron)
#Evaluate happiness stats of schedule
aaron_hap = stats.sched_happiness(df_aaron, aaron_sched)
print('Total Happiness: ', aaron_hap[0])
print()
print('Availability to happiness correlation: ', aaron_hap[1])
print()
print('Variance of happiness: ', aaron_hap[2])
print()
print('Envy stats: ', aaron_hap[3])
print()
print('Incorrect stats: ', aaron_hap[4])
print()

Total Happiness:  0.9767441860465116

Availability to happiness correlation:  0.2534326003778107

Variance of happiness:  0.7909090909090911

Envy stats:  43

Incorrect stats:  6



In [7]:
df_max_weight = pd.DataFrame(sheet.get_all_records())

#create graph nodes and weight edges
graph_data = max_weight_cap.create_graph(df_max_weight)
student_nodes = graph_data[0]
slot_nodes = graph_data[1]
wt = graph_data[2]
student_cap = graph_data[3]
slot_cap = graph_data[4]

#solve the problem and get the ordered schedule
p = max_weight_cap.solve_wbm(student_nodes, slot_nodes, wt, student_cap, slot_cap)
unordered_sched_dict = max_weight_cap.get_solution(p)
max_weight_sched = max_weight_cap.order_sched(df_max_weight, unordered_sched_dict)
# Schedule.print_sched(max_weight_sched)
labTA.schedule_to_df(df_max_weight, max_weight_sched)

# #make list of overlap students and resolve them
# overlaps = max_weight_cap.get_overlaps(df_max_weight, max_weight_sched)
# max_weight_cap.resolve_overlaps(df_max_weight, max_weight_sched, overlaps)

#print df and happiness stats
# print(df_max_weight)

max_weight_hap = stats.sched_happiness(df_max_weight, max_weight_sched)
print('Total Happiness: ', max_weight_hap[0])
print()
print('Availability to happiness correlation: ', max_weight_hap[1])
print()
print('Variance of happiness: ', max_weight_hap[2])
print()
print('Envy stats: ', max_weight_hap[3])
print()
print('Incorrect stats: ', max_weight_hap[4])
print()

/Users/aaronlichtblau/opt/miniconda3/lib/python3.7/site-packages/pulp/pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Total Happiness:  0.9922480620155039

Availability to happiness correlation:  0.45586699410583836

Variance of happiness:  1.173737373737374

Envy stats:  40

Incorrect stats:  4



In [ ]:
#demo swap function
#use exp stats to find student we want to swap
#print df
#get suggested swap
#make swap and print new df

In [ ]:
#demo weighing function
#print df
#weigh student-slot who has high score